In [3]:
import os
import glob
import xarray as xr
import netCDF4 as nc
import numpy as np
from pyproj import Proj, Transformer, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import math
import pandas as pd
import dask.array as da
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units

## Read netcdf files

In [131]:
current_dir = os.getcwd()

# WFDE5 data
wfde5_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/WFDE5/'
psurf_path = glob.glob(wfde5_path + 'psurf/*.nc')
qair_path = glob.glob(wfde5_path + 'qair/*.nc')
rainf_path = glob.glob(wfde5_path + 'rainf/*.nc')
swdown_path = glob.glob(wfde5_path + 'swdown/*.nc')
tair_path = glob.glob(wfde5_path + 'tair/*.nc')
wind_path = glob.glob(wfde5_path + 'wind/*.nc')
cell_data_path = glob.glob(wfde5_path + 'cell_data/*.nc')
asurf_path = glob.glob(wfde5_path + 'asurf/*.nc')

# Discharge data
discharge_path = glob.glob(current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/discharge/*.txt')

# Testdata
test_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/'
discharge_test_path = test_path + 'discharge.nc'
relhum_test_path = test_path + 'relative_humidity.nc'
precip_test_path = test_path + 'precipitation.nc'
swdown_test_path = test_path + 'radiation.nc'
temp_test_path = test_path + 'temperature.nc'
wind_test_path = test_path + 'wind_speed.nc'
cell_data_test_path = test_path + 'cell_data.nc'

In [142]:
# Datasets from the WDFE5 data
psurf = xr.open_mfdataset(psurf_path)
qair = xr.open_mfdataset(qair_path)
rainf = xr.open_mfdataset(rainf_path)
swdown = xr.open_mfdataset(swdown_path)
tair = xr.open_mfdataset(tair_path)
wind = xr.open_mfdataset(wind_path)
cell_data = xr.open_mfdataset(cell_data_path)
asurf = xr.open_mfdataset(asurf_path)

# Discharge dataset
#discharge = pd.read_csv(discharge_path[0], )

# Datsets from nidelva data (on Shyft format)
discharge_test = xr.open_mfdataset(discharge_test_path)
relhum_test = xr.open_mfdataset(relhum_test_path)
precip_test = xr.open_mfdataset(precip_test_path)
swdown_test = xr.open_mfdataset(swdown_test_path)
temp_test = xr.open_mfdataset(temp_test_path)
wind_test = xr.open_mfdataset(wind_test_path)
cell_data_test = xr.open_mfdataset(cell_data_test_path)

In [188]:
discharge = pd.read_csv(discharge_path[0], header= 16)
discharge = discharge.rename(columns = {' 2015 ' : 'daily_flow'})
discharge.index = pd.to_datetime(discharge.index)
discharge_raw_ds = discharge.to_xarray()

## Get variables

In [86]:
# Make dataset where lat, lon have the same dimensions (here such that longitudes match the latitudes)

asurf = asurf.sel(lon = asurf.lon.values[11:58])
#wind = wind.sel(lon = wind.lon.values[11:58])
#psurf = psurf.sel(lon = psurf.lon.values[11:58])
#qair = qair.sel(lon = qair.lon.values[11:58])
#rainf = rainf.sel(lon = rainf.lon.values[11:58])
#swdown = swdown.sel(lon = swdown.lon.values[11:58])
#tair = tair.sel(lon = tair.lon.values[11:58])



In [87]:
## Update values and do conversions

### Temperature
tair_degC = tair.Tair - 273.15
#tair = tair.update({'Tair' : (['time', 'lon', 'lat'], tair_degC.values)})

### Relative humidity using metPy package

#### Convert pressure from Pa to hPa

psurf_hpa = psurf.PSurf * 0.01

### From rainfall (kg m-2 s-2) to precipitation (mm)

precipitation_mm = rainf.Rainf * 60 * 60


In [39]:
n = int(len(psurf.PSurf) / 6)
tn_1 = tair.isel(time = slice(0, n)).time.values
tn_2 = tair.isel(time = slice(n, 2 * n)).time.values
tn_3 = tair.isel(time = slice(2 * n, 2 * n + n)).time.values
tn_4 = tair.isel(time = slice(3 * n, 3 * n + n)).time.values
tn_5 = tair.isel(time = slice(4 * n, 4 * n + n)).time.values
tn_6 = tair.isel(time = slice(5 * n, 5 * n + n)).time.values


In [46]:
rel_hum_1 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_1).values * units.hPa, tair_degC.sel(time = tn_1).values * units.degC, qair.Qair.sel(time = tn_1).values).to('percent').magnitude

In [40]:
rel_hum_2 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_2).values * units.hPa, tair_degC.sel(time = tn_2).values * units.degC, qair.Qair.sel(time = tn_2).values).to('percent').magnitude

In [41]:
rel_hum_3 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_3).values * units.hPa, tair_degC.sel(time = tn_3).values * units.degC, qair.Qair.sel(time = tn_3).values).to('percent').magnitude

In [42]:
rel_hum_4 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_4).values * units.hPa, tair_degC.sel(time = tn_4).values * units.degC, qair.Qair.sel(time = tn_4).values).to('percent').magnitude

In [43]:
rel_hum_5 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_5).values * units.hPa, tair_degC.sel(time = tn_5).values * units.degC, qair.Qair.sel(time = tn_5).values).to('percent').magnitude

In [44]:
rel_hum_6 = relative_humidity_from_specific_humidity(psurf_hpa.sel(time = tn_6).values * units.hPa, tair_degC.sel(time = tn_6).values * units.degC, qair.Qair.sel(time = tn_6).values).to('percent').magnitude

In [58]:
rel_hum = np.append(rel_hum_1, rel_hum_2).reshape(2 * n, 47, 47)
rel_hum = np.append(rel_hum, rel_hum_3).reshape(3 * n, 47, 47)
rel_hum = np.append(rel_hum, rel_hum_4).reshape(4 * n, 47, 47)
rel_hum = np.append(rel_hum, rel_hum_5).reshape(5 * n, 47, 47)
rel_hum = np.append(rel_hum, rel_hum_6).reshape(6 * n, 47, 47)

In [61]:
## Extract lat and lon

lon = asurf.lon.values
lat = asurf.lat.values

In [62]:
## Find UTM CRS
utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=np.min(lon),
        south_lat_degree=np.min(lat),
        east_lon_degree=np.max(lon),
        north_lat_degree=np.max(lat),
    ),
)

utm_crs = CRS.from_epsg(utm_crs_list[0].code)
utm_crs

<Derived Projected CRS: EPSG:32642>
Name: WGS 84 / UTM zone 42N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 66°E and 72°E, northern hemisphere between equator and 84°N, onshore and offshore. Afghanistan. India. Kazakhstan. Kyrgyzstan. Pakistan. Russian Federation. Tajikistan. Uzbekistan.
- bounds: (66.0, 0.0, 72.0, 84.0)
Coordinate Operation:
- name: UTM zone 42N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [104]:
## Making a pyproj object for UTM Zone 41N

crs_4326 = CRS.from_epsg(4326)
crs_32642 = CRS.from_epsg(32642)

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_4326, crs_32642, always_xy = True)

x, y = transformer.transform(lon, lat)

In [192]:
asurf_utm = asurf.assign_coords(coords = dict(x = x, y = y))
#wind_utm = wind.assign_coords(coords = dict(x = x, y = y))
#psurf_utm = psurf.assign_coords(coords = dict(x = x, y = y))
#qair_utm = qair.assign_coords(coords = dict(x = x, y = y))
#rainf_utm = rainf.assign_coords(coords = dict(x = x, y = y))
#swdown_utm = swdown.assign_coords(coords = dict(x = x, y = y))
#tair_utm = tair.assign_coords(coords = dict(x = x, y = y))
disc_utm = discharge_raw_ds.assign_coords(coords=dict(x = x, y = y))
#disc_utm = disc_utm.expand_dims(dict = (time = len(disc_utm.index), series = ))

#wind_utm = wind_utm.update({'Wind' : (['time', 'x', 'y'], wind_utm.Wind.values)})
#psurf_utm = psurf_utm.update({'PSurf' : (['time', 'x', 'y'], psurf_utm.PSurf.values)})
#rel_hum_utm = qair_utm.update({'relative_humidity' : (['time', 'x', 'y'], rel_hum)})
#precipitation_utm = rainf_utm.update({'precipitation' : (['time', 'x', 'y'], precipitation_mm.values )})
#swdown_utm = swdown_utm.update({'SWdown' : (['time', 'x', 'y'], swdown_utm.SWdown.values)})
#tair_utm = tair_utm.update({'Tair' : (['time', 'x', 'y'], tair_utm.Tair.values)})
disc_utm = disc_utm.update({'discharge' : (['time', 'x', 'y'], disc_utm.values)})

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['time', 'x', 'y'], <bound method Mapping.values of <xarray.Dataset>
Dimensions:     (index: 5654, x: 47, y: 47)
Coordinates:
  * index       (index) datetime64[ns] 2000-01-01 2000-01-02 ... 2015-12-31
  * x           (x) float64 2.595e+05 3.135e+05 ... 2.267e+06 2.298e+06
  * y           (y) float64 1.798e+06 1.853e+06 ... 4.489e+06 4.556e+06
Data variables:
    daily_flow  (index) float64 39.1 39.1 38.5 38.5 38.3 ... 26.2 27.3 27.2 27.0>) to Variable.

In [194]:
disc_utm

<xarray.Dataset>
Dimensions:     (index: 5654, x: 47, y: 47)
Coordinates:
  * index       (index) datetime64[ns] 2000-01-01 2000-01-02 ... 2015-12-31
  * x           (x) float64 2.595e+05 3.135e+05 ... 2.267e+06 2.298e+06
  * y           (y) float64 1.798e+06 1.853e+06 ... 4.489e+06 4.556e+06
Data variables:
    daily_flow  (index) float64 39.1 39.1 38.5 38.5 38.3 ... 26.2 27.3 27.2 27.0

In [193]:
discharge_test

<xarray.Dataset>
Dimensions:       (time: 1096, series: 6)
Coordinates:
  * time          (time) datetime64[ns] 2012-12-31T23:00:00 ... 2015-12-31T23...
    x             (series) float64 dask.array<chunksize=(6,), meta=np.ndarray>
    y             (series) float64 dask.array<chunksize=(6,), meta=np.ndarray>
    z             (series) float64 dask.array<chunksize=(6,), meta=np.ndarray>
Dimensions without coordinates: series
Data variables:
    series_name   (series) object dask.array<chunksize=(6,), meta=np.ndarray>
    catchment_id  (series) int32 dask.array<chunksize=(6,), meta=np.ndarray>
    crs           int32 ...
    discharge     (time, series) float64 dask.array<chunksize=(1096, 6), meta=np.ndarray>

In [108]:
precipitation_utm

<xarray.Dataset>
Dimensions:        (lat: 47, lon: 47, time: 262968, x: 47, y: 47)
Coordinates:
  * lat            (lat) float64 16.25 16.75 17.25 17.75 ... 38.25 38.75 39.25
  * lon            (lon) float64 66.75 67.25 67.75 68.25 ... 88.75 89.25 89.75
  * time           (time) datetime64[ns] 1990-01-01 ... 2019-12-31T23:00:00
  * x              (x) float64 2.595e+05 3.135e+05 ... 2.267e+06 2.298e+06
  * y              (y) float64 1.798e+06 1.853e+06 ... 4.489e+06 4.556e+06
Data variables:
    Rainf          (time, lat, lon) float32 dask.array<chunksize=(744, 47, 47), meta=np.ndarray>
    precipitation  (time, x, y) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
Attributes:
    title:        WATCH Forcing Data methodology applied to ERA5 data
    institution:  Copernicus Climate Change Service
    contact:      http://copernicus-support.ecmwf.int
    comment:      Methodology implementation for ERA5 and dataset production ...
    Conventions:  CF-1.7
    summary:      ERA5 data regridded to half degree regular lat-lon; Genuine...
    reference:    Cucchi et al., 2020, Earth Syst. Sci. Data, 12(3), 2097–212...
    licence:      The dataset is distributed under the Licence to Use Coperni...

In [109]:
#stacked_wind = wind_utm.Wind.stack(station = ['x', 'y'])
#stacked_psurf = psurf_utm.PSurf.stack(station = ['x', 'y'])
#stacked_rel_hum = rel_hum_utm.relative_humidity.stack(station = ['x', 'y'])
stacked_precipitation = precipitation_utm.precipitation.stack(station = ['x', 'y'])
#stacked_swdown = swdown_utm.SWdown.stack(station = ['x', 'y'])
#stacked_tair = tair_utm.Tair.stack(station = ['x', 'y'])

#stacked_wind_values = stacked_wind.values
#stacked_psurf_values = stacked_psurf.values
#stacked_rel_hum_values = stacked_rel_hum.values
stacked_precipitation_values = stacked_precipitation.values
#stacked_swdown_values = stacked_swdown.values
#stacked_tair_values = stacked_tair.values

In [110]:
#stacked_wind_station = stacked_wind.assign_coords(station = ('station', np.arange(len(stacked_wind.station))))
#stacked_psurf_station = stacked_psurf.assign_coords(station = ('station', np.arange(len(stacked_psurf.station))))
#stacked_rel_hum_station = stacked_rel_hum.assign_coords(station = ('station', np.arange(len(stacked_rel_hum.station))))
stacked_precipitation_station = stacked_precipitation.assign_coords(station = ('station', np.arange(len(stacked_precipitation.station))))
#stacked_swdown_station = stacked_swdown.assign_coords(station = ('station', np.arange(len(stacked_swdown.station))))
#stacked_tair_station = stacked_tair.assign_coords(station = ('station', np.arange(len(stacked_tair.station))))


In [111]:
#time_wind = stacked_wind.time.values
#time_tair = stacked_tair.time.values
#time_swdown = stacked_swdown.time.values
#time_rel_hum = stacked_rel_hum.time.values
time_precipitation = stacked_precipitation.time.values

#x_wind = stacked_wind.x.values.astype('float64')
#y_wind = stacked_wind.y.values.astype('float64')
# x_tair = stacked_tair.x.values.astype('float64')
# y_tair = stacked_tair.y.values.astype('float64')
#x_swdown = stacked_swdown.x.values.astype('float64')
#y_swdown = stacked_swdown.y.values.astype('float64')
# x_rel_hum = stacked_rel_hum.x.values.astype('float64')
# y_rel_hum = stacked_rel_hum.y.values.astype('float64')
x_precipitation = stacked_precipitation.x.values.astype('float64')
y_precipitation = stacked_precipitation.y.values.astype('float64')

# Get elevation from each xy-point
z = asurf.ASurf.stack(z_xy = ['lon', 'lat']).values.astype('float64')

#station_wind = stacked_wind_station.station.values.astype('object')
#station_tair = stacked_tair_station.station.values.astype('object')
#station_swdown = stacked_swdown_station.station.values.astype('object')
#station_rel_hum = stacked_rel_hum_station.station.values.astype('object')
station_precipitation = stacked_precipitation_station.station.values.astype('object')

#wind_speed = stacked_wind_station.values.astype('float64')
#pressure = stacked_psurf_station.values.astype('float64')
#rel_hum = stacked_rel_hum_station.values.astype('float64')
precipitation = stacked_precipitation_station.values.astype('float64')
#radiation = stacked_swdown_station.values.astype('float64')
#temp = stacked_tair_station.values.astype('float64')

crs = np.array(-2147483647).astype('int32')


In [112]:
#x_wind_da = da.from_array(x_wind, chunks= len(x_wind))
#y_wind_da = da.from_array(y_wind, chunks = len(y_wind))
#x_tair_da = da.from_array(x_tair, chunks= len(x_tair))
#y_tair_da = da.from_array(y_tair, chunks = len(y_tair))
# x_swdown_da = da.from_array(x_swdown, chunks= len(x_swdown))
# y_swdown_da = da.from_array(y_swdown, chunks = len(y_swdown))
# x_rel_hum_da = da.from_array(x_rel_hum, chunks= len(x_rel_hum))
# y_rel_hum_da = da.from_array(y_rel_hum, chunks = len(y_rel_hum))
x_precipitation_da = da.from_array(x_precipitation, chunks= len(x_precipitation))
y_precipitation_da = da.from_array(y_precipitation, chunks = len(y_precipitation))

z_da = da.from_array(z, chunks = len(z))

#series_name_wind_da = da.from_array(station_wind, chunks = len(station_wind))
#series_name_tair_da = da.from_array(station_tair, chunks = len(station_tair))
#series_name_swdown_da = da.from_array(station_swdown, chunks = len(station_swdown))
#series_name_rel_hum_da = da.from_array(station_rel_hum, chunks = len(station_rel_hum))
series_name_precipitation_da = da.from_array(station_precipitation, chunks = len(station_precipitation))

#wind_speed_da = da.from_array(wind_speed, chunks = len(wind_speed))
#pressure_da = da.from_array(pressure, chunks = len(pressure))
#rel_hum_da = da.from_array(rel_hum, chunks = len(rel_hum))
precipitation_da = da.from_array(precipitation, chunks = len(precipitation))
#radiation_da = da.from_array(radiation, chunks = len(radiation))
#temp_da = da.from_array(temp, chunks = len(temp))


In [113]:
# wind_ds = xr.Dataset(
    
#     data_vars = dict(
#         series_name = (['station'], series_name_da),
#         crs = ([], crs),
#         wind_speed = (['time', 'station'], wind_speed_da)
#     ),
#     coords = dict(
#         time = (['time'], time_wind),
#         x = (['station'], x_wind_da),
#         y = (['station'], y_wind_da),
#         z = (['station'], z_da)
#     ),
   
# )

# temp_ds = xr.Dataset(
    
#     data_vars = dict(
#         series_name = (['station'], series_name_da),
#         crs = ([], crs),
#         temperature = (['time', 'station'], temp_da)
#     ),
#     coords = dict(
#         time = (['time'], time_tair),
#         x = (['station'], x_tair_da),
#         y = (['station'], y_tair_da),
#         z = (['station'], z_da)
#     ),
   
# )

# radiation_ds = xr.Dataset(
    
#     data_vars = dict(
#         series_name = (['station'], series_name_swdown_da),
#         crs = ([], crs),
#         global_radiation = (['time', 'station'], radiation_da)
#     ),
#     coords = dict(
#         time = (['time'], time_swdown),
#         x = (['station'], x_swdown_da),
#         y = (['station'], y_swdown_da),
#         z = (['station'], z_da)
#     ), 
# )

# rel_hum_ds = xr.Dataset(
    
#     data_vars = dict(
#         series_name = (['station'], series_name_rel_hum_da),
#         crs = ([], crs),
#         relative_humidity = (['time', 'station'], rel_hum_da)
#     ),
#     coords = dict(
#         time = (['time'], time_rel_hum),
#         x = (['station'], x_rel_hum_da),
#         y = (['station'], y_rel_hum_da),
#         z = (['station'], z_da)
#     ), 
# )

precipitation_ds = xr.Dataset(
    
    data_vars = dict(
        series_name = (['station'], series_name_precipitation_da),
        crs = ([], crs),
        precipitation = (['time', 'station'], precipitation_da)
    ),
    coords = dict(
        time = (['time'], time_precipitation),
        x = (['station'], x_precipitation_da),
        y = (['station'], y_precipitation_da),
        z = (['station'], z_da)
    ), 
)

In [114]:
precipitation_ds

<xarray.Dataset>
Dimensions:        (station: 2209, time: 262968)
Coordinates:
  * time           (time) datetime64[ns] 1990-01-01 ... 2019-12-31T23:00:00
    x              (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    y              (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    z              (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
Dimensions without coordinates: station
Data variables:
    series_name    (station) object dask.array<chunksize=(2209,), meta=np.ndarray>
    crs            int32 -2147483647
    precipitation  (time, station) float64 dask.array<chunksize=(262968, 2209), meta=np.ndarray>

In [75]:
rel_hum_ds

<xarray.Dataset>
Dimensions:            (station: 2209, time: 262968)
Coordinates:
  * time               (time) datetime64[ns] 1990-01-01 ... 2019-12-31T23:00:00
    x                  (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    y                  (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    z                  (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
Dimensions without coordinates: station
Data variables:
    series_name        (station) object dask.array<chunksize=(2209,), meta=np.ndarray>
    crs                int32 -2147483647
    relative_humidity  (time, station) float64 dask.array<chunksize=(262968, 2209), meta=np.ndarray>

In [39]:
radiation_ds.global_radiation

<xarray.DataArray 'global_radiation' (time: 262968, station: 2209)>
dask.array<array, shape=(262968, 2209), dtype=float64, chunksize=(262968, 2209), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1990-01-01 ... 2019-12-31T23:00:00
    x        (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(2209,), meta=np.ndarray>
Dimensions without coordinates: station

In [40]:
swdown_test.global_radiation

<xarray.DataArray 'global_radiation' (time: 8760, station: 1)>
dask.array<open_dataset-dedf7304830e7cf8af82171d6b6f6792global_radiation, shape=(8760, 1), dtype=float64, chunksize=(8760, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2013-09-01 ... 2014-08-31T23:00:00
    x        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: station
Attributes:
    units:         mm
    grid_mapping:  crs

In [116]:
# WIND

# wind_ds.x.attrs['axis'] = 'X'
# wind_ds.x.attrs['standard_name'] = 'projection_x_coordinate'
# wind_ds.x.attrs['units'] = 'm'

# wind_ds.y.attrs['axis'] = 'Y'
# wind_ds.y.attrs['standard_name'] = 'projection_y_coordinate'
# wind_ds.y.attrs['units'] = 'm'

# wind_ds.z.attrs['units'] = 'm'
# wind_ds.z.attrs['standard_name'] = 'height'
# wind_ds.z.attrs['axis'] = 'Z'
# wind_ds.z.attrs['long_name'] = 'height above mean sea level'

# wind_ds.series_name.attrs['cf_role'] = 'timeseries_id'

# wind_ds.wind_speed.attrs['units'] = 'mm'
# wind_ds.wind_speed.attrs['grid_mapping'] = 'crs'

# wind_ds.crs.attrs['proj'] = '+proj=utm +zone=41 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
# wind_ds.crs.attrs['grid_mapping_name'] = 'transverse_mercator'
# wind_ds.crs.attrs['epsg_code'] = 'EPSG:32642'


# TEMPERATURE

# temp_ds.x.attrs['axis'] = 'X'
# temp_ds.x.attrs['standard_name'] = 'projection_x_coordinate'
# temp_ds.x.attrs['units'] = 'm'

# temp_ds.y.attrs['axis'] = 'Y'
# temp_ds.y.attrs['standard_name'] = 'projection_y_coordinate'
# temp_ds.y.attrs['units'] = 'm'

# temp_ds.z.attrs['units'] = 'm'
# temp_ds.z.attrs['standard_name'] = 'height'
# temp_ds.z.attrs['axis'] = 'Z'
# temp_ds.z.attrs['long_name'] = 'height above mean sea level'

# temp_ds.series_name.attrs['cf_role'] = 'timeseries_id'

# temp_ds.wind_speed.attrs['units'] = 'mm'
# temp_ds.wind_speed.attrs['grid_mapping'] = 'crs'

# temp_ds.crs.attrs['proj'] = '+proj=utm +zone=41 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
# temp_ds.crs.attrs['grid_mapping_name'] = 'transverse_mercator'
# temp_ds.crs.attrs['epsg_code'] = 'EPSG:32642'

# RADIATION

# radiation_ds.x.attrs['axis'] = 'X'
# radiation_ds.x.attrs['standard_name'] = 'projection_x_coordinate'
# radiation_ds.x.attrs['units'] = 'm'

# radiation_ds.y.attrs['axis'] = 'Y'
# radiation_ds.y.attrs['standard_name'] = 'projection_y_coordinate'
# radiation_ds.y.attrs['units'] = 'm'

# radiation_ds.z.attrs['units'] = 'm'
# radiation_ds.z.attrs['standard_name'] = 'height'
# radiation_ds.z.attrs['axis'] = 'Z'
# radiation_ds.z.attrs['long_name'] = 'height above mean sea level'

# radiation_ds.series_name.attrs['cf_role'] = 'timeseries_id'

# radiation_ds.global_radiation.attrs['units'] = 'mm'
# radiation_ds.global_radiation.attrs['grid_mapping'] = 'crs'

# radiation_ds.crs.attrs['proj'] = '+proj=utm +zone=41 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
# radiation_ds.crs.attrs['grid_mapping_name'] = 'transverse_mercator'
# radiation_ds.crs.attrs['epsg_code'] = 'EPSG:32642'

# RELATIVE HUMIDITY

# rel_hum_ds.x.attrs['axis'] = 'X'
# rel_hum_ds.x.attrs['standard_name'] = 'projection_x_coordinate'
# rel_hum_ds.x.attrs['units'] = 'm'

# rel_hum_ds.y.attrs['axis'] = 'Y'
# rel_hum_ds.y.attrs['standard_name'] = 'projection_y_coordinate'
# rel_hum_ds.y.attrs['units'] = 'm'

# rel_hum_ds.z.attrs['units'] = 'm'
# rel_hum_ds.z.attrs['standard_name'] = 'height'
# rel_hum_ds.z.attrs['axis'] = 'Z'
# rel_hum_ds.z.attrs['long_name'] = 'height above mean sea level'

# rel_hum_ds.series_name.attrs['cf_role'] = 'timeseries_id'

# rel_hum_ds.relative_humidity.attrs['units'] = 'mm'
# rel_hum_ds.relative_humidity.attrs['grid_mapping'] = 'crs'

# rel_hum_ds.crs.attrs['proj'] = '+proj=utm +zone=41 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
# rel_hum_ds.crs.attrs['grid_mapping_name'] = 'transverse_mercator'
# rel_hum_ds.crs.attrs['epsg_code'] = 'EPSG:32642'

# RELATIVE HUMIDITY

precipitation_ds.x.attrs['axis'] = 'X'
precipitation_ds.x.attrs['standard_name'] = 'projection_x_coordinate'
precipitation_ds.x.attrs['units'] = 'm'

precipitation_ds.y.attrs['axis'] = 'Y'
precipitation_ds.y.attrs['standard_name'] = 'projection_y_coordinate'
precipitation_ds.y.attrs['units'] = 'm'

precipitation_ds.z.attrs['units'] = 'm'
precipitation_ds.z.attrs['standard_name'] = 'height'
precipitation_ds.z.attrs['axis'] = 'Z'
precipitation_ds.z.attrs['long_name'] = 'height above mean sea level'

precipitation_ds.series_name.attrs['cf_role'] = 'timeseries_id'

precipitation_ds.precipitation.attrs['units'] = 'mm'
precipitation_ds.precipitation.attrs['grid_mapping'] = 'crs'

precipitation_ds.crs.attrs['proj'] = '+proj=utm +zone=41 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
precipitation_ds.crs.attrs['grid_mapping_name'] = 'transverse_mercator'
precipitation_ds.crs.attrs['epsg_code'] = 'EPSG:32642'

In [45]:
swdown_test.global_radiation

<xarray.DataArray 'global_radiation' (time: 8760, station: 1)>
dask.array<open_dataset-dedf7304830e7cf8af82171d6b6f6792global_radiation, shape=(8760, 1), dtype=float64, chunksize=(8760, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2013-09-01 ... 2014-08-31T23:00:00
    x        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    y        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    z        (station) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: station
Attributes:
    units:         mm
    grid_mapping:  crs